In [ ]:
%matplotlib inline

from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *


provider = IBMQ.load_account()
simulator=Aer.get_backend('qasm_simulator')

qi=QuantumRegister(1)
qs=QuantumRegister(2)
qxs=QuantumRegister(2)
qb=QuantumRegister(1)
qo=QuantumRegister(4)
c=ClassicalRegister(4)

circuit=QuantumCircuit(qi,qb,qs,qxs,qo,c)

def fun_or(qc,q0,q1,q2):
    qc.x(q0)
    qc.x(q1)
    qc.ccx(q0,q1,q2)
    qc.x(q2)
    qc.x(q1)
    qc.x(q0)

def or3(qc,q0,q1,q2,b,q3):
    fun_or(qc,q0,q1,b)
    fun_or(qc,b,q2,q3)
    qc.reset(b)

def fun_and(qc,q0,q1,q2):
    qc.ccx(q0,q1,q2)

def and3(qc,q0,q1,q2,b,q3):
    fun_and(qc,q0,q1,b)
    fun_and(qc,b,q2,q3)
    qc.reset(b)

def or4(qc,q0,q1,q2,q3,b1,b2,q4):
    or3(qc,q0,q1,q2,b1,b2)
    fun_or(qc,b2,q3,q4)
    qc.reset(b1)
    qc.reset(b2)

#Input
circuit.x(qi[0])

#Control signal
circuit.x(qs[1])

for i in range(0,2):
    circuit.cx(qs[i],qxs[i])
    circuit.x(qxs[i])

and3(circuit,qxs[0],qxs[1],qi[0],qb[0],qo[0])
and3(circuit,qs[0],qxs[1],qi[0],qb[0],qo[1])
and3(circuit,qxs[0],qs[1],qi[0],qb[0],qo[2])
and3(circuit,qs[0],qs[1],qi[0],qb[0],qo[3])

circuit.measure(qo[0],c[3])
circuit.measure(qo[1],c[2])
circuit.measure(qo[2],c[1])
circuit.measure(qo[3],c[0])

job=execute(circuit,simulator,shots=100)
result=job.result()
counts=result.get_counts(circuit)

print(counts)
#circuit.draw()
#plot_histogram(counts)
